In [1]:
import datetime
import itertools
import numpy as np
from qiskit import Aer, IBMQ, QuantumCircuit, transpile
from mzm_generation import KitaevHamiltonianTask, run_task, save

In [2]:
# get backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
hardware_backend = provider.get_backend('ibmq_manila')
simulator_backend = Aer.get_backend("statevector_simulator")

In [3]:
# set parameters

# number of modes
n_modes = 3

# x values
num = 20
x_min = 1e-7
x_max = 3.0
x_values = np.linspace(x_min, x_max, num=num)

# backend
backend = hardware_backend

# number of shots
shots = 10000

# experiment id used for saving data
experiment_id = datetime.datetime.now().isoformat()
if backend is simulator_backend:
    experiment_id += '_sim'

In [4]:
# generate tasks
tasks = []
for chemical_potential in x_values:
    for n_particles in range(n_modes + 1):
        for occupied_orbitals in itertools.combinations(range(n_modes), n_particles):
            task = KitaevHamiltonianTask(
                experiment_id=experiment_id,
                n_modes=n_modes,
                tunneling=-1.0,
                superconducting=1.0,
                chemical_potential=chemical_potential,
                occupied_orbitals=occupied_orbitals,
                shots=shots)
            tasks.append(task)

In [5]:
# run jobs
jobs = {}
for task in tasks:
    jobs[task] = run_task(backend, task)

In [6]:
# save results
for task, job in jobs.items():
    data = job.result().get_counts()
    save(task, data)